In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages

In [2]:
# Import our input dataset
mortgage_df = pd.read_csv('Final Project Data.csv')
mortgage_df.head()
len(mortgage_df.index)

297452

In [3]:
# Drop the unwanted variables from the data
mortgage_input = mortgage_df.drop(columns = ['Loan Identifier'])
mortgage_input.head()

,Origination Channel,Seller Name,Original Interest Rate,Original UPB,Original Loan Term,Origination Date,First Payment Date,Original Loan-to-value (LTV),Original Combined Loan-to-value (CLTV),Number of Borrowers,...,Property Type,Number of Units,Occupancy Type,Property State,Zip Code Short,Primary Mortgage Insurance Percent,Product Type,Co-borrower Credit Score at Origination,Mortgage Insurance Type,Relocation Mortgage Indicator
0,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",6,324000,360,2018-09-01,2018-11-01,80,80,2,...,PU,1,P,CA,925,NaN,FRM,665.0,NaN,N
1,B,OTHER,5,307000,360,2018-12-01,2019-02-01,90,90,1,...,PU,1,P,TX,770,25.0,FRM,NaN,1.0,N
2,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5,256000,360,2018-11-01,2019-01-01,90,90,2,...,SF,1,S,NC,286,25.0,FRM,738.0,1.0,N
3,C,"WELLS FARGO BANK, N.A.",5,248000,360,2018-12-01,2019-02-01,90,90,2,...,SF,1,P,IL,600,25.0,FRM,791.0,1.0,N
4,B,"UNITED SHORE FINANCIAL SERVICES, LLC DBA UNITE...",4,490000,360,2019-03-01,2019-05-01,67,67,1,...,CO,1,P,CA,945,NaN,FRM,NaN,NaN,N


In [4]:
# Drop NA rows
mortgage_input = mortgage_input.dropna()
len(mortgage_input.index)
mortgage_input.head()

,Origination Channel,Seller Name,Original Interest Rate,Original UPB,Original Loan Term,Origination Date,First Payment Date,Original Loan-to-value (LTV),Original Combined Loan-to-value (CLTV),Number of Borrowers,...,Property Type,Number of Units,Occupancy Type,Property State,Zip Code Short,Primary Mortgage Insurance Percent,Product Type,Co-borrower Credit Score at Origination,Mortgage Insurance Type,Relocation Mortgage Indicator
2,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5,256000,360,2018-11-01,2019-01-01,90,90,2,...,SF,1,S,NC,286,25.0,FRM,738.0,1.0,N
3,C,"WELLS FARGO BANK, N.A.",5,248000,360,2018-12-01,2019-02-01,90,90,2,...,SF,1,P,IL,600,25.0,FRM,791.0,1.0,N
6,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5,264000,360,2018-10-01,2018-12-01,95,95,2,...,PU,1,P,TX,773,30.0,FRM,778.0,1.0,Y
8,R,OTHER,5,174000,360,2018-12-01,2019-02-01,97,97,2,...,SF,1,P,IN,479,35.0,FRM,708.0,1.0,N
9,B,"UNITED SHORE FINANCIAL SERVICES, LLC DBA UNITE...",5,204000,360,2019-01-01,2019-03-01,95,95,2,...,PU,1,P,UT,844,25.0,FRM,722.0,1.0,N


In [5]:
# Generate our categorical variable list
mortgage_cat = mortgage_input.dtypes[mortgage_input.dtypes == "object"].index.tolist()
mortgage_cat

['Origination Channel',
 'Seller Name',
 'Origination Date',
 'First Payment Date',
 'First Time Home Buyer Indicator',
 'Loan Purpose ',
 'Property Type',
 'Occupancy Type',
 'Property State',
 'Product Type',
 'Relocation Mortgage Indicator']

In [6]:
# Check the number of unique values in each column
mortgage_input[mortgage_cat].nunique()

Origination Channel                 3
Seller Name                        17
Origination Date                   19
First Payment Date                 19
First Time Home Buyer Indicator     2
Loan Purpose                        3
Property Type                       5
Occupancy Type                      3
Property State                     52
Product Type                        1
Relocation Mortgage Indicator       2
dtype: int64

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mortgage_input[mortgage_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(mortgage_cat)
encode_df.head()

,Origination Channel_B,Origination Channel_C,Origination Channel_R,"Seller Name_AMERIHOME MORTGAGE COMPANY, LLC",Seller Name_DITECH FINANCIAL LLC,"Seller Name_EAGLE HOME MORTGAGE, LLC",Seller Name_FAIRWAY INDEPENDENT MORTGAGE CORPORATION,"Seller Name_FLAGSTAR BANK, FSB",Seller Name_FREEDOM MORTGAGE CORP.,"Seller Name_JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",...,Property State_UT,Property State_VA,Property State_VT,Property State_WA,Property State_WI,Property State_WV,Property State_WY,Product Type_FRM,Relocation Mortgage Indicator_N,Relocation Mortgage Indicator_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
mortgage_merged = mortgage_input.merge(encode_df,left_index=True, right_index=True)
mortgage_merged = mortgage_merged.drop(mortgage_cat,1)
mortgage_merged.head()

,Original Interest Rate,Original UPB,Original Loan Term,Original Loan-to-value (LTV),Original Combined Loan-to-value (CLTV),Number of Borrowers,Original Debt to Income Ratio,Borrower Credit Score at Origination,Number of Units,Zip Code Short,...,Property State_UT,Property State_VA,Property State_VT,Property State_WA,Property State_WI,Property State_WV,Property State_WY,Product Type_FRM,Relocation Mortgage Indicator_N,Relocation Mortgage Indicator_Y
2,5,256000,360,90,90,2,41.0,728.0,1,286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,5,248000,360,90,90,2,40.0,730.0,1,600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,5,264000,360,95,95,2,45.0,710.0,1,773,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
8,5,174000,360,97,97,2,37.0,747.0,1,479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
9,5,204000,360,95,95,2,42.0,717.0,1,844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [9]:
def decision(probability):
    return np.random.choice([0, 1], size = len(mortgage_merged.index), p = [1 - probability, probability])

foreclosure_rng = decision(0.266)
foreclosure_rng

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
mortgage_merged["Foreclosure"] = foreclosure_rng
mortgage_merged

,Original Interest Rate,Original UPB,Original Loan Term,Original Loan-to-value (LTV),Original Combined Loan-to-value (CLTV),Number of Borrowers,Original Debt to Income Ratio,Borrower Credit Score at Origination,Number of Units,Zip Code Short,...,Property State_VA,Property State_VT,Property State_WA,Property State_WI,Property State_WV,Property State_WY,Product Type_FRM,Relocation Mortgage Indicator_N,Relocation Mortgage Indicator_Y,Foreclosure
2,5,256000,360,90,90,2,41.0,728.0,1,286,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
3,5,248000,360,90,90,2,40.0,730.0,1,600,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
6,5,264000,360,95,95,2,45.0,710.0,1,773,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
8,5,174000,360,97,97,2,37.0,747.0,1,479,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
9,5,204000,360,95,95,2,42.0,717.0,1,844,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46281,5,209000,360,95,95,2,33.0,753.0,1,180,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
46287,5,109000,360,96,96,2,35.0,807.0,1,338,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
46289,5,105000,360,97,97,2,47.0,683.0,1,74,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
46294,6,228000,360,97,101,2,47.0,752.0,1,891,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = mortgage_merged["Foreclosure"].values
X = mortgage_merged.drop(["Foreclosure"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [14]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [15]:
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,1
4,0,0
5,0,0
6,0,0
7,0,1
8,0,1
9,0,0


In [16]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.75178866263071


In [17]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 8
activ_func = "tanh"

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activ_func)
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activ_func))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                3360      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 200       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 3,569
Trainable params: 3,569
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/cp.{acc:.4f}-{epoch:02d}.hdf5"

In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='accuracy',
    verbose=1,
    #save_best_only=True,
    save_weights_only=False,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
5280/5448 [============================>.] - ETA: 0s - loss: 0.6574 - acc: 0.6223
Epoch 00001: saving model to checkpoints/cp.0.6244-01.hdf5
5448/5448 [==============================] - 1s 201us/sample - loss: 0.6562 - acc: 0.6244
Epoch 2/100
5248/5448 [===========================>..] - ETA: 0s - loss: 0.5902 - acc: 0.7214
Epoch 00002: saving model to checkpoints/cp.0.7223-02.hdf5
5448/5448 [==============================] - 0s 53us/sample - loss: 0.5895 - acc: 0.7223
Epoch 3/100
4704/5448 [========================>.....] - ETA: 0s - loss: 0.5744 - acc: 0.7309
Epoch 00003: saving model to checkpoints/cp.0.7307-03.hdf5
5448/5448 [==============================] - 0s 51us/sample - loss: 0.5757 - acc: 0.7307
Epoch 4/100
5184/5448 [===========================>..] - ETA: 0s - loss: 0.5670 - acc: 0.7330
Epoch 00004: saving model to checkpoints/cp.0.7326-04.hdf5
5448/5448 [===============

5376/5448 [============================>.] - ETA: 0s - loss: 0.3648 - acc: 0.8486
Epoch 00033: saving model to checkpoints/cp.0.8477-33.hdf5
5448/5448 [==============================] - 0s 42us/sample - loss: 0.3660 - acc: 0.8477
Epoch 34/100
5120/5448 [===========================>..] - ETA: 0s - loss: 0.3603 - acc: 0.8512
Epoch 00034: saving model to checkpoints/cp.0.8493-34.hdf5
5448/5448 [==============================] - 0s 44us/sample - loss: 0.3599 - acc: 0.8493
Epoch 35/100
4160/5448 [=====================>........] - ETA: 0s - loss: 0.3552 - acc: 0.8483
Epoch 00035: saving model to checkpoints/cp.0.8484-35.hdf5
5448/5448 [==============================] - 0s 41us/sample - loss: 0.3548 - acc: 0.8484
Epoch 36/100
5344/5448 [============================>.] - ETA: 0s - loss: 0.3504 - acc: 0.8516
Epoch 00036: saving model to checkpoints/cp.0.8524-36.hdf5
5448/5448 [==============================] - 0s 42us/sample - loss: 0.3501 - acc: 0.8524
Epoch 37/100
5312/5448 [=================

Epoch 00066: saving model to checkpoints/cp.0.9077-66.hdf5
5448/5448 [==============================] - 0s 47us/sample - loss: 0.2445 - acc: 0.9077
Epoch 67/100
4960/5448 [==========================>...] - ETA: 0s - loss: 0.2376 - acc: 0.9091
Epoch 00067: saving model to checkpoints/cp.0.9075-67.hdf5
5448/5448 [==============================] - 0s 46us/sample - loss: 0.2419 - acc: 0.9075
Epoch 68/100
4640/5448 [========================>.....] - ETA: 0s - loss: 0.2359 - acc: 0.9123
Epoch 00068: saving model to checkpoints/cp.0.9110-68.hdf5
5448/5448 [==============================] - 0s 49us/sample - loss: 0.2388 - acc: 0.9110
Epoch 69/100
5376/5448 [============================>.] - ETA: 0s - loss: 0.2352 - acc: 0.9113
Epoch 00069: saving model to checkpoints/cp.0.9106-69.hdf5
5448/5448 [==============================] - 0s 42us/sample - loss: 0.2357 - acc: 0.9106
Epoch 70/100
4992/5448 [==========================>...] - ETA: 0s - loss: 0.2302 - acc: 0.9133
Epoch 00070: saving model to

Epoch 100/100
4128/5448 [=====================>........] - ETA: 0s - loss: 0.1756 - acc: 0.9411
Epoch 00100: saving model to checkpoints/cp.0.9407-100.hdf5
5448/5448 [==============================] - 0s 40us/sample - loss: 0.1758 - acc: 0.9407
1817/1817 - 0s - loss: 1.3006 - acc: 0.6258
Loss: 1.3005778135410953, Accuracy: 0.6257567405700684


Just commited this, my first draft at a neural network. Does some real rough encoding of categories, scaling, etc.
Looks like it's being overfitted or something atm, need to figure that out.

In [20]:
X_train_scaled

array([[-2.40084126, -0.28255216,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092],
       [-0.21942312, -0.37415897,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092],
       [-0.21942312,  0.80007377,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092],
       ...,
       [-0.21942312,  0.44197442,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092],
       [-0.21942312, -1.09868555,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092],
       [-0.21942312,  0.30872815,  0.10832533, ...,  0.        ,
         0.08041092, -0.08041092]])